In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
train = pd.read_csv('../data/train.csv', parse_dates=['date'])
test = pd.read_csv('../data/test.csv', parse_dates=['date'])
stores = pd.read_csv('../data/stores.csv')
oil = pd.read_csv('../data/oil.csv', parse_dates=['date'])
holidays = pd.read_csv('../data/holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv('../data/transactions.csv', parse_dates=['date'])

In [3]:
def prepare_data(train_df, stores_df, oil_df, holidays_df):
    df = train_df.merge(stores_df, on='store_nbr', how='left')
    
    oil_df['dcoilwtico'] = oil_df['dcoilwtico'].ffill()
    df = df.merge(oil_df, on='date', how='left')
    
    nat_holidays = holidays_df[(holidays_df['locale'] == 'National') & 
                               (holidays_df['transferred'] == False)]
    nat_holidays = nat_holidays.drop_duplicates('date')[['date', 'type']]
    nat_holidays = nat_holidays.rename(columns={'type': 'holiday_type'})
    
    df = df.merge(nat_holidays, on='date', how='left')
    df['is_holiday'] = df['holiday_type'].notnull().astype(int)
    
    return df

df = prepare_data(train, stores, oil, holidays)

In [4]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,1
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,1
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,1
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,1


In [5]:
def create_features(df):
    df['day_of_week'] = df['date'].dt.dayofweek
    df['day_of_month'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    df['is_payday'] = ((df['day_of_month'] == 15) | (df.date.dt.is_month_end)).astype(int)
    
    # Lags: What was the sale 16 days ago? 
    # (We use 16 because the test set is 15 days long)
    df['lag_16'] = df.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.shift(16))
    df['lag_30'] = df.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.shift(30))
    
    # Rolling Mean: What was the average of the 7 days ending 16 days ago?
    df['rolling_mean_7'] = df.groupby(['store_nbr', 'family'])['lag_16'].transform(lambda x: x.rolling(7).mean())
    
    return df

df = create_features(df)

In [6]:
# We'll train on everything before August 2017
# And validate on the first 15 days of August 2017
train_set = df[df['date'] < '2017-08-01'].dropna()
val_set = df[(df['date'] >= '2017-08-01') & (df['date'] <= '2017-08-15')]

# Define features and target
features = ['store_nbr', 'onpromotion', 'dcoilwtico', 'is_holiday', 
            'day_of_week', 'is_payday', 'lag_16', 'lag_30', 'rolling_mean_7']
target = 'sales'

X_train, y_train = train_set[features], train_set[target]
X_val, y_val = val_set[features], val_set[target]

In [7]:
# Apply Log Transformation to the target
# This helps the model focus on percentage errors (better for RMSLE)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print("Target transformed to log scale.")

Target transformed to log scale.


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Use a small number of trees for a quick test
model = RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1)
model.fit(X_train, y_train)

# Predict and Calculate RMSLE (the competition metric)
preds = model.predict(X_val)
preds = np.maximum(0, preds) # Sales cannot be negative

score = np.sqrt(mean_squared_log_error(y_val, preds))
print(f"Baseline RMSLE: {score}")

Baseline RMSLE: 0.7902125653221613


In [9]:
# 1. Prepare Test set similarly to Train set
# We need to concatenate them to calculate lags correctly for the test period
full_df = pd.concat([train, test], axis=0).reset_index(drop=True)

# 2. Re-apply the merging and feature logic
# (Using the functions we defined earlier)
full_df = prepare_data(full_df, stores, oil, holidays)
full_df = create_features(full_df)

# 3. Separate them back out
train_final = full_df[full_df['sales'].notnull()]
test_final = full_df[full_df['sales'].isnull()]

print(f"Final training rows: {len(train_final)}")
print(f"Final test rows (should be 28512): {len(test_final)}")

Final training rows: 3000888
Final test rows (should be 28512): 28512


In [10]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# 1. We'll store our results here
all_test_preds = []
all_test_ids = []

# 2. Get the list of all product families (e.g., 'BEVERAGES', 'MEATS', etc.)
families = train_final['family'].unique()

print(f"Starting Per-Family Training for {len(families)} families...")

for fam in families:
    # Filter the data for JUST this family
    train_fam = train_final[train_final['family'] == fam]
    test_fam = test_final[test_final['family'] == fam]
    
    # Define X (features) and y (target)
    X_train = train_fam[features]
    
    # --- THE TRICK ---
    # We use np.log1p to turn sales into a log scale. 
    # This is the secret to a better RMSLE score!
    y_train = np.log1p(train_fam['sales']) 
    
    X_test_fam = test_fam[features]
    
    # 3. Train a model specifically for this family
    model = RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42)
    model.fit(X_train, y_train)
    
    # 4. Predict and convert back from log scale using np.expm1
    fam_preds = np.expm1(model.predict(X_test_fam))
    
    # Store predictions and the matching IDs
    all_test_preds.extend(fam_preds)
    all_test_ids.extend(test_fam['id'])
    
    print(f"✅ Finished: {fam}")

Starting Per-Family Training for 33 families...
✅ Finished: AUTOMOTIVE
✅ Finished: BABY CARE
✅ Finished: BEAUTY
✅ Finished: BEVERAGES
✅ Finished: BOOKS
✅ Finished: BREAD/BAKERY
✅ Finished: CELEBRATION
✅ Finished: CLEANING
✅ Finished: DAIRY
✅ Finished: DELI
✅ Finished: EGGS
✅ Finished: FROZEN FOODS
✅ Finished: GROCERY I
✅ Finished: GROCERY II
✅ Finished: HARDWARE
✅ Finished: HOME AND KITCHEN I
✅ Finished: HOME AND KITCHEN II
✅ Finished: HOME APPLIANCES
✅ Finished: HOME CARE
✅ Finished: LADIESWEAR
✅ Finished: LAWN AND GARDEN
✅ Finished: LINGERIE
✅ Finished: LIQUOR,WINE,BEER
✅ Finished: MAGAZINES
✅ Finished: MEATS
✅ Finished: PERSONAL CARE
✅ Finished: PET SUPPLIES
✅ Finished: PLAYERS AND ELECTRONICS
✅ Finished: POULTRY
✅ Finished: PREPARED FOODS
✅ Finished: PRODUCE
✅ Finished: SCHOOL AND OFFICE SUPPLIES
✅ Finished: SEAFOOD


In [11]:
# 5. Create the final submission file
submission_v2 = pd.DataFrame({
    'id': np.array(all_test_ids).astype(int),
    'sales': np.array(all_test_preds)
})

# Sort by ID to make sure it's in the right order for Kaggle
submission_v2 = submission_v2.sort_values('id')

submission_v2.to_csv('../submissions/submission_per_family.csv', index=False)
print("\n🚀 New submission file saved: ../submissions/submission_per_family.csv")


🚀 New submission file saved: ../submissions/submission_per_family.csv
